![image](https://drive.google.com/u/0/uc?id=15DUc09hFGqR8qcpYiN1OajRNaASmiL6d&export=download)

# **Taller No. 9 - ISIS4825**

## **Regresión Logística para Clasificación Binaria y Análisis de Componentes Principales**
## **Contenido**
1. [**Objetivos**](#id1)
2. [**Problema**](#id2)
3. [**Importando las librerías necesarias para el laboratorio**](#id3)
4. [**Visualización y Análisis Exploratorio**](#id4)
5. [**Preparación de los Datos**](#id5)
6. [**Modelamiento**](#id6)
7. [**Predicción**](#id7)
8. [**Validación**](#id8)
9. [**Trabajo Asíncrono**](#id9)

## **Objetivos**<a name="id1"></a>

- Familiarizarse con la regresón logística y sus parámetros.
- Aplicar PCA como técnica de reducción de la dimensión.
- Pensar en un problema de categorías desbalanceadas.

## **Problema**<a name="id2"></a>
- En un dataset de imágenes de lenguaje de señas buscamos clasificar todas las imágenes que pertenezcan a la letra R y a la letra E.

### **Notebook Configuration**

In [ ]:
!shred -u setup_colab.py
!wget -q "https://github.com/jpcano1/python_utils/raw/main/setup_colab_general.py" -O setup_colab_general.py
!wget -q "https://github.com/jpcano1/python_utils/raw/main/ISIS_4825/setup_colab.py" -O setup_colab.py
import setup_colab as setup
setup.setup_workshop_9()

## **Importando las librerías necesarias para el laboratorio**<a name="id3"></a>

In [ ]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import (train_test_split, ShuffleSplit, 
                                     cross_val_score, GridSearchCV)
from sklearn.metrics import (precision_score, recall_score, confusion_matrix, 
                             accuracy_score, f1_score, roc_curve, 
                             precision_recall_curve)
from sklearn.utils import resample

from utils import general as gen

from tqdm.auto import tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use("ggplot")
import seaborn as sns

### **Carga de Datos**

In [ ]:
train_dir = gen.create_and_verify("data", "sign_mnist_train.csv")
test_dir = gen.create_and_verify("data", "sign_mnist_test.csv")

In [ ]:
train_data = pd.read_csv(train_dir)
test_data = pd.read_csv(test_dir)

In [ ]:
full_X_train, full_y_train = train_data.drop(columns=["label"]), train_data["label"]
X_test, y_test = test_data.drop(columns=["label"]), test_data["label"]

In [ ]:
if (full_y_train == 9).sum() == 0 or (y_test == 9).sum() == 0:
    full_y_train = full_y_train.apply(lambda x: x - 1 if x > 8 else x)
    y_test = y_test.apply(lambda x: x - 1 if x > 8 else x)

In [ ]:
if train_data.query("label==9").size == 0 or test_data.query("label==9").size == 0:
    train_data["label"] = train_data["label"].apply(lambda x: x - 1 if x > 8 else x)
    test_data["label"] = test_data["label"].apply(lambda x: x - 1 if x > 8 else x)

In [ ]:
full_X_train.shape, full_y_train.shape

In [ ]:
X_test.shape, y_test.shape

## **Visualización y Análisis Exploratorio**<a name="id4"></a>
- En este laboratorio vamos a usar una variación del dataset de MNIST que utiliza imágenes de lenguaje de señas. Cada número, desde el 0 hasta el 23, será asignado a cada seña.

![image](https://docs.google.com/uc?export=download&id=17e0VoyKW_0HgmItDWqGiyB1Bl6C-e8G8)

- Este dataset consta de 7172 imágenes en total con clases madianamente balanceadas.

In [ ]:
random_sample = full_X_train.sample(9, random_state=1234)
gen.visualize_subplot(
    random_sample.values.reshape(-1, 28, 28),
    full_y_train[random_sample.index].values, (3, 3), (6, 6)
)

In [ ]:
plt.figure(figsize=(10, 6))
ax = full_y_train.plot(kind="hist", bins=24)
ax.set_xticks(range(24))
plt.show()

In [ ]:
full_y_train.value_counts().sort_index()

## **Preparación de los Datos**<a name="id5"></a>

In [ ]:
random_sample = full_X_train.sample(1, random_state=5678)
sample_target = full_y_train[random_sample.index].values

In [ ]:
gen.imshow(random_sample.values.reshape(28, 28), color=False, 
           title=sample_target[0])

In [ ]:
train_data_a = train_data.query("(label==4) | (label==16)")
test_data_a = test_data.query("(label==4) | (label==16)")

In [ ]:
full_X_train, full_y_train = train_data_a.drop(columns=["label"]), train_data_a["label"]
X_test, y_test = test_data_a.drop(columns=["label"]), test_data_a["label"]

In [ ]:
full_y_train = (full_y_train == 4).astype("uint8")
y_test = (y_test == 4).astype("uint8")

### **Train Set, Validation Set, Test Set**

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(full_X_train,
                                                  full_y_train,
                                                  test_size=0.2,
                                                  random_state=1234)

In [ ]:
random_sample = X_train.sample(9, random_state=1234)
gen.visualize_subplot(
    random_sample.values.reshape(-1, 28, 28),
    y_train[random_sample.index].values, (3, 3), (6, 6)
)

### **Desbalanceo de Clases**
- Algo muy usual dentro de los problemas de clasificación binaria es el desbalanceo de clases. Esto implicará que nuestro algoritmo probablemente clasificará la clase mayoritaria, lo cual no debería ser.

$$\text{freq}_p = \frac{\text{Número de instancias positivas}}{\text{Total de instancias}}$$

$$\text{freq}_n = \frac{\text{Número de instancias negativas}}{\text{Total de instancias}}$$

- En este caso, vamos a usar un método de la librería `Scikit-Learn` llamado `resample`. Esta función nos va a permitir hacer un remuestreo de la clase con mayor cantidad de instancias, para que tenga igual número de muestras que la clase con menor cantidad de instancias.

In [ ]:
plt.figure(figsize=(10, 6))
y_train.astype("int").plot(kind="hist")
plt.show()

In [ ]:
def cat_frequencies(labels):
    freq_p = labels.mean()
    freq_n = 1 - freq_p
    return freq_p, freq_n

In [ ]:
f_p, f_n = cat_frequencies(y_train)

In [ ]:
f_p, f_n

In [ ]:
total = pd.concat([X_train, y_train], axis=1)

In [ ]:
neg_class = total.query("label==0")
pos_class = total.query("label==1")

In [ ]:
neg_class_downsampled = resample(neg_class, replace=False,
                                 n_samples=len(pos_class), 
                                 random_state=1234)

In [ ]:
balanced_data = pd.concat([neg_class_downsampled, pos_class])

In [ ]:
y_train = balanced_data["label"]
X_train = balanced_data.drop("label", axis=1)

In [ ]:
f_p, f_n = cat_frequencies(y_train)

In [ ]:
f_p, f_n

## **Modelamiento**<a name="id6"></a>
- En esta ocasión se hará uso de la regresión Logística, una función lineal ampliamente usada en problemas de clasificación binaria.

$$\text{sigmoid}(x)=\frac{1}{1+e^{-x}}$$

In [ ]:
log_reg = LogisticRegression(n_jobs=-1)

In [ ]:
log_reg.fit(X_train, y_train)

## **Predicción**<a name="id7"></a>

In [ ]:
random_sample = X_test.sample(9, random_state=1234)
y_pred = log_reg.predict(random_sample)

In [ ]:
gen.visualize_subplot(
    random_sample.values.reshape(-1, 28, 28),
    y_pred, (3, 3), (6, 6)
)

## **Validación**<a name="id8"></a>

In [ ]:
y_pred = log_reg.predict(X_test)

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
pd.DataFrame(conf_matrix)

In [ ]:
plt.matshow(conf_matrix, cmap="gray")
plt.grid(0)
plt.show()

In [ ]:
norm_conf_mat = conf_matrix / conf_matrix.sum(axis=1, keepdims=True)
np.fill_diagonal(norm_conf_mat, 0)

In [ ]:
plt.matshow(norm_conf_mat, cmap="gray")
plt.grid(0)
plt.show()

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

In [ ]:
cross_val_score(log_reg, full_X_train, full_y_train, cv=4, scoring="accuracy")

## **Trabajo Asíncrono**<a name="id9"></a>
1. En primera instancia, utilizar [`GridSearch`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) para determinar los mejores valores de los hiperparámetros. Para eso, averiguar sobre los siguientes hiperparámetros:

    - `penalty: 'l1' y 'l2'`
    - `C`
    - `solver: 'saga', 'liblinear'`
2. Luego, realizar una clasificación multiclase sobre este mismo dataset y con las clases desde la 'A' hasta la 'K'. Revisar la documentación de la regresión logística [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
3. Por último, aplicar [PCA](https://es.wikipedia.org/wiki/An%C3%A1lisis_de_componentes_principales) para construir un conjunto de datos de dimensión reducida para el problema multiclase del punto 2. Comparar los resultados con base en las métricas de clasificación sobre el conjunto de test. Para lo anterior, se puede hacer uso de la clase [`PCA`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) de `scikit-learn`